In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import math
import os
import copy
import time

from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from sklearn.cluster import KMeans

import torch
import math
from torch import nn  # nn module contains CrossEntropyLoss
from torch import optim  # optim module contains SGD and lr_scheduler


In [3]:
songString = "CCGGAAGFFEEDDCGGFFEEDGGFFEEDCCGGAAGFFEEDDC"
notes=list(unique(list(songString)))
print(notes)
chord=[[0],
       [1],
       [2],
       [3],
       [4],
       [5]]
print(chord)
DaisyBell=list()
DaisyBellMatrix=[]
for note in list(songString):
    row=[0]*6
    row[notes.index(note)]=1
    DaisyBellMatrix.append(row)
    DaisyBell.append(chord[notes.index(note)])
print(DaisyBell)
print(len(DaisyBell))
DaisyBellMatrix=numpy.array(DaisyBellMatrix)

['A', 'C', 'D', 'E', 'F', 'G']
[[0], [1], [2], [3], [4], [5]]
[[1], [1], [5], [5], [0], [0], [5], [4], [4], [3], [3], [2], [2], [1], [5], [5], [4], [4], [3], [3], [2], [5], [5], [4], [4], [3], [3], [2], [1], [1], [5], [5], [0], [0], [5], [4], [4], [3], [3], [2], [2], [1]]
42


In [4]:
HotDogsongString = "AACCGGGFFFFDDGCCFFDDDAAFFCCDCCGGGGGFFEEDDC"
notes=list(unique(list(HotDogsongString)))
chord=[[0],
       [1],
       [2],
       [3],
       [4],
       [5]]
HotDog=list()
HotDogMatrix=[]
for note in list(HotDogsongString):
    row=[0]*6
    row[notes.index(note)]=1
    HotDogMatrix.append(row)
    HotDog.append(chord[notes.index(note)])
print(HotDog)
HotDogMatrix=numpy.array(HotDogMatrix)

[[0], [0], [1], [1], [5], [5], [5], [4], [4], [4], [4], [2], [2], [5], [1], [1], [4], [4], [2], [2], [2], [0], [0], [4], [4], [1], [1], [2], [1], [1], [5], [5], [5], [5], [5], [4], [4], [3], [3], [2], [2], [1]]


In [5]:
HotDog#DaisyBell

[[0],
 [0],
 [1],
 [1],
 [5],
 [5],
 [5],
 [4],
 [4],
 [4],
 [4],
 [2],
 [2],
 [5],
 [1],
 [1],
 [4],
 [4],
 [2],
 [2],
 [2],
 [0],
 [0],
 [4],
 [4],
 [1],
 [1],
 [2],
 [1],
 [1],
 [5],
 [5],
 [5],
 [5],
 [5],
 [4],
 [4],
 [3],
 [3],
 [2],
 [2],
 [1]]

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
song3 = "CDEGAGEDCDEGAGEDCDEGAGEDCDEGAGEDCDEGAG"
notes=list(unique(list(song3)))
chord=[[0],
       [1],
       [2],
       [3],
       [4],
       [5]]
song=list()
songMatrix=[]
for note in list(song3):
    row=[0]*6
    row[notes.index(note)]=1
    songMatrix.append(row)
    song.append(chord[notes.index(note)])
print(song)
songMatrix=numpy.array(songMatrix)

[[1], [2], [3], [4], [0], [4], [3], [2], [1], [2], [3], [4], [0], [4], [3], [2], [1], [2], [3], [4], [0], [4], [3], [2], [1], [2], [3], [4], [0], [4], [3], [2], [1], [2], [3], [4], [0], [4]]


In [7]:
# Store the songs in a list
#songs = [DaisyBell, HotDog, song]
songs = [DaisyBell]

In [8]:
def get_batch(source, i, song_index, bptt=42):
    data=[]
    target=[]
    for pos in range(i,i+bptt):
        data.append(source[song_index][pos%len(source[song_index])])
        target+=source[song_index][(pos+1)%len(source[song_index])]
    return data, target

In [12]:
bptt=42
for i in range(42):
    d,t=get_batch(songs,i,0)
    print(f"input {d}")
    print(f"output {t}")
    print(t[:4])
    print("\n")


input [[1], [1], [5], [5], [0], [0], [5], [4], [4], [3], [3], [2], [2], [1], [5], [5], [4], [4], [3], [3], [2], [5], [5], [4], [4], [3], [3], [2], [1], [1], [5], [5], [0], [0], [5], [4], [4], [3], [3], [2], [2], [1]]
output [1, 5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 5, 5, 4, 4, 3, 3, 2, 5, 5, 4, 4, 3, 3, 2, 1, 1, 5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 1]
[1, 5, 5, 0]


input [[1], [5], [5], [0], [0], [5], [4], [4], [3], [3], [2], [2], [1], [5], [5], [4], [4], [3], [3], [2], [5], [5], [4], [4], [3], [3], [2], [1], [1], [5], [5], [0], [0], [5], [4], [4], [3], [3], [2], [2], [1], [1]]
output [5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 5, 5, 4, 4, 3, 3, 2, 5, 5, 4, 4, 3, 3, 2, 1, 1, 5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 1, 1]
[5, 5, 0, 0]


input [[5], [5], [0], [0], [5], [4], [4], [3], [3], [2], [2], [1], [5], [5], [4], [4], [3], [3], [2], [5], [5], [4], [4], [3], [3], [2], [1], [1], [5], [5], [0], [0], [5], [4], [4], [3], [3], [2], [2], [1], [1], [1]]
output [5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 5, 5, 4, 4

In [11]:
class TransformerModel(nn.Module):
    def __init__(self, ntoken, d_model, nhead, d_hid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead, d_hid, dropout), nlayers)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor, verbose=False) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        if verbose:
            print(src.shape)
            figure()
            imshow(src.detach().numpy().reshape((42,20)))
        src = self.pos_encoder(src)
        if verbose:
            print(src.shape)
            figure()
            imshow(src.detach().numpy().reshape((42,20)))
        output = self.transformer_encoder(src, src_mask)
        #print(output.shape)
        #self.retainer=output[-1].copy()
        if verbose:
            print(output.shape)
            figure()
            imshow(output.detach().numpy().reshape((42,20)))
        output = self.decoder(output)
        if verbose:
            print(output.shape)
            figure()
            imshow(output.detach().numpy().reshape((42,6)))
        return output

        
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [12]:
# I generated this part not sure if its right
model = TransformerModel(ntoken=6, d_model=64, nhead=4, d_hid=256, nlayers=2, dropout=0.2)
print(model)

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (encoder): Embedding(6, 64)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=256, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=256, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=64, out_features=6, bias=True)
)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [13]:
model.state_dict()

OrderedDict([('pos_encoder.pe',
              tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
                         0.0000e+00,  1.0000e+00]],
              
                      [[ 8.4147e-01,  5.4030e-01,  6.8156e-01,  ...,  1.0000e+00,
                         1.3335e-04,  1.0000e+00]],
              
                      [[ 9.0930e-01, -4.1615e-01,  9.9748e-01,  ...,  1.0000e+00,
                         2.6670e-04,  1.0000e+00]],
              
                      ...,
              
                      [[ 9.5625e-01, -2.9254e-01,  6.4315e-01,  ...,  6.3049e-01,
                         6.1813e-01,  7.8608e-01]],
              
                      [[ 2.7050e-01, -9.6272e-01, -5.1133e-02,  ...,  6.3036e-01,
                         6.1823e-01,  7.8599e-01]],
              
                      [[-6.6395e-01, -7.4778e-01, -7.1816e-01,  ...,  6.3022e-01,
                         6.1834e-01,  7.8591e-01]]])),
             ('encoder.weight',
            

In [14]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)


In [28]:
def train(model, songs, optimizer, criterion, bptt=42, device='cpu'):
    model.train()
    total_loss = 0.
    total_items = 0

    for song_index, song in enumerate(songs):
        for i in range(0, len(song) - 1, bptt):
            data, targets = get_batch(songs, i, song_index, bptt)
            data_tensor = torch.tensor(data, dtype=torch.long).to(device)
            targets_tensor = torch.tensor(targets, dtype=torch.long).to(device)
            src_mask = generate_square_subsequent_mask(bptt).to(device)

            optimizer.zero_grad()
            output = model(data_tensor, src_mask)
            loss = criterion(output.view(-1, ntokens), targets_tensor)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * data_tensor.size(0)
            total_items += data_tensor.size(0)

    avg_loss = total_loss / total_items
    ppl = math.exp(avg_loss)
    print(f'Training Loss: {avg_loss:.2f} | Training Perplexity: {ppl:.2f}')


def evaluate(model, songs, criterion, bptt=42, device='cpu'):
    model.eval()
    total_loss = 0.
    with torch.no_grad():
        for song_index, song in enumerate(songs):
            for i in range(0, len(song) - 1, bptt):
                data, targets = get_batch(songs, i, song_index, bptt)
                data_tensor = torch.tensor(data, dtype=torch.long).to(device)
                targets_tensor = torch.tensor(targets, dtype=torch.long).to(device)
                src_mask = generate_square_subsequent_mask(bptt).to(device)

                output = model(data_tensor, src_mask)
                total_loss += len(data) * criterion(output.view(-1, ntokens), targets_tensor).item()

    avg_loss = total_loss / sum(len(song) for song in songs)
    return avg_loss


# Parameters setup
bptt: int = 42  # sequence length
ntokens = 6 # number of unique notes
emsize = 20 # embedding dimension
nhid = 20 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4 # the number of heads in the multiheadattention models
dropout = 0.0 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
epochs = 2000 # The number of epochs
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Training loop
for epoch in range(1, epochs + 1):
    train(model, songs, optimizer, criterion)
    # Evaluate on the validation dataset
    val_loss = evaluate(model, songs, criterion)  # Replace `songs` with your validation dataset if available
    
    # Calculate and print the validation perplexity
    val_ppl = math.exp(val_loss)
    print(f'| End of epoch {epoch:3d} | Validation Loss: {val_loss:.2f} | Validation Perplexity: {val_ppl:8.2f}')


Training Loss: 1.84 | Training Perplexity: 6.29
| End of epoch   1 | Validation Loss: 1.82 | Validation Perplexity:     6.20
Training Loss: 1.82 | Training Perplexity: 6.20
| End of epoch   2 | Validation Loss: 1.81 | Validation Perplexity:     6.10
Training Loss: 1.81 | Training Perplexity: 6.10
| End of epoch   3 | Validation Loss: 1.79 | Validation Perplexity:     6.02
Training Loss: 1.79 | Training Perplexity: 6.02
| End of epoch   4 | Validation Loss: 1.78 | Validation Perplexity:     5.93
Training Loss: 1.78 | Training Perplexity: 5.93
| End of epoch   5 | Validation Loss: 1.77 | Validation Perplexity:     5.85
Training Loss: 1.77 | Training Perplexity: 5.85
| End of epoch   6 | Validation Loss: 1.75 | Validation Perplexity:     5.78
Training Loss: 1.75 | Training Perplexity: 5.78
| End of epoch   7 | Validation Loss: 1.74 | Validation Perplexity:     5.70
Training Loss: 1.74 | Training Perplexity: 5.70
| End of epoch   8 | Validation Loss: 1.73 | Validation Perplexity:     5.63


/var/folders/87/pnb9p7_n30s29gprj1gbqbbm0000gn/T/ipykernel_67881/1964148146.py:41: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  avg_loss = total_loss / sum(len(song) for song in songs)


| End of epoch  67 | Validation Loss: 0.90 | Validation Perplexity:     2.46
Training Loss: 0.90 | Training Perplexity: 2.46
| End of epoch  68 | Validation Loss: 0.89 | Validation Perplexity:     2.43
Training Loss: 0.89 | Training Perplexity: 2.43
| End of epoch  69 | Validation Loss: 0.88 | Validation Perplexity:     2.41
Training Loss: 0.88 | Training Perplexity: 2.41
| End of epoch  70 | Validation Loss: 0.87 | Validation Perplexity:     2.38
Training Loss: 0.87 | Training Perplexity: 2.38
| End of epoch  71 | Validation Loss: 0.86 | Validation Perplexity:     2.36
Training Loss: 0.86 | Training Perplexity: 2.36
| End of epoch  72 | Validation Loss: 0.85 | Validation Perplexity:     2.34
Training Loss: 0.85 | Training Perplexity: 2.34
| End of epoch  73 | Validation Loss: 0.84 | Validation Perplexity:     2.31
Training Loss: 0.84 | Training Perplexity: 2.31
| End of epoch  74 | Validation Loss: 0.83 | Validation Perplexity:     2.29
Training Loss: 0.83 | Training Perplexity: 2.29


In [32]:
for i ,song in enumerate(songs):
    for j in range(bptt):
        data,target=get_batch(songs,j, i)
        # Convert the input notes to a tensor and add an extra dimension

        # Create a mask for the input
        src_mask = generate_square_subsequent_mask(len(data)).to(device)

        # Run the model on the input tensor
        output = model(torch.tensor(data,dtype=torch.long),src_mask)

        # Detach the output from the computation graph and convert to numpy array
        output_array = output.detach().numpy()

        # Get the predicted notes by finding the index of the maximum value in each output vector
        predicted_notes = np.argmax(output_array, axis=-1)

        predicted_notes_list = [note for sublist in predicted_notes for note in sublist]  # Flattening the list

        # Calculate accuracy
        accuracy = np.mean(np.array(predicted_notes_list) == np.array(target))
        print(f'Accuracy for song {i + 1}, index {j}: {accuracy:.2f}')
        print(f'target notes:    {target}')
        print(f'predicted notes: {predicted_notes_list}')


Accuracy for song 1, index 0: 1.00
target notes:    [1, 5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 5, 5, 4, 4, 3, 3, 2, 5, 5, 4, 4, 3, 3, 2, 1, 1, 5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 1]
predicted notes: [1, 5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 5, 5, 4, 4, 3, 3, 2, 5, 5, 4, 4, 3, 3, 2, 1, 1, 5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 1]
Accuracy for song 1, index 1: 0.48
target notes:    [5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 5, 5, 4, 4, 3, 3, 2, 5, 5, 4, 4, 3, 3, 2, 1, 1, 5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 1, 1]
predicted notes: [1, 5, 5, 5, 0, 4, 3, 4, 3, 2, 1, 1, 5, 5, 5, 4, 4, 3, 3, 5, 5, 5, 4, 4, 3, 3, 1, 1, 1, 4, 5, 3, 3, 5, 4, 4, 3, 3, 2, 2, 1, 1]
Accuracy for song 1, index 2: 0.21
target notes:    [5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 5, 5, 4, 4, 3, 3, 2, 5, 5, 4, 4, 3, 3, 2, 1, 1, 5, 5, 0, 0, 5, 4, 4, 3, 3, 2, 2, 1, 1, 1, 5]
predicted notes: [0, 0, 3, 3, 3, 3, 3, 2, 2, 1, 1, 2, 2, 2, 2, 4, 2, 3, 5, 5, 5, 5, 4, 3, 3, 3, 3, 1, 4, 4, 3, 3, 0, 4, 4, 4, 3, 3, 3, 4, 1, 1]
Accuracy for song 1, index 3: 0